# Step 1: Load spaCy language model
The code starts by importing the necessary libraries and modules. The spaCy library is used for natural language processing tasks. It loads a pre-trained English language model called "en_core_web_sm," which is a small English pipeline that includes tokenization, part-of-speech tagging, dependency parsing, and named entity recognition.

In [17]:
pip install spacy scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import spacy
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score


In [19]:
nlp = spacy.load("en_core_web_sm")

# Step 2: Process the text data
The preprocess_text function takes a text input and processes it using the loaded spaCy language model. It tokenizes the text into words, performs lemmatization (reducing words to their base or dictionary form), and then joins the lemmatized words back into a single string. This step helps to normalize the text data and reduce the complexity of the language used in the text.

In [20]:
def preprocess_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join(token.lemma_ for token in doc)
    return lemmatized_text

# Step 3: Load the data and preprocess it
The load_data function takes a directory path as input. It iterates over subdirectories corresponding to different labels such as "spam" or "ham" within the provided directory. For each label, it reads the text files, processes the text using the preprocess_text function, and appends the processed text to the texts list and the corresponding label to the labels list. This function is used to load and preprocess the training and testing data.

In [21]:
def load_data(dir_path):
    texts = []
    labels = []
    for label in os.listdir(dir_path):
        label_path = os.path.join(dir_path, label)
        for filename in os.listdir(label_path):
            file_path = os.path.join(label_path, filename)
            with open(file_path, "r", encoding="latin-1") as file:
                text = file.read()
                processed_text = preprocess_text(text)
                texts.append(processed_text)
                labels.append(label)
    return texts, labels


train_texts, train_labels = load_data("A-2-Corpus-Naive-Bayes-and-Logistic-Regression/train")
test_texts, test_labels = load_data("A-2-Corpus-Naive-Bayes-and-Logistic-Regression/test")


# Step 4: Feature Extraction (Bag-of-Words)
The code initializes a CountVectorizer object from scikit-learn. This vectorizer is used to convert the processed text data into a bag-of-words representation. The bag-of-words model represents a document as a vector where each element corresponds to the frequency of a particular word in the document. The fit_transform method of the vectorizer is applied to the training data (train_texts) to generate the feature matrix X_train, and the transform method is applied to the test data (test_texts) to generate the feature matrix X_test.

In [22]:

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)


# Step 5: Multinomial Naive Bayes classifier
A Multinomial Naive Bayes classifier (MultinomialNB) from scikit-learn is initialized. Naive Bayes is a classification algorithm based on Bayes' theorem, and the "multinomial" variant is suitable for text classification tasks like this. The classifier is trained using the training feature matrix X_train and corresponding labels (train_labels) using the fit method.

In [23]:
clf = MultinomialNB()
clf.fit(X_train, train_labels)

MultinomialNB()

# Step 6: Evaluation
The trained Naive Bayes classifier is used to predict labels for the test feature matrix X_test, and the predictions are stored in the predictions array. The code then calculates the accuracy and precision of the classifier's predictions compared to the actual test labels. The accuracy_score and precision_score functions from scikit-learn are used for this purpose. Finally, the accuracy and precision values are printed to the console.

In [24]:

predictions = clf.predict(X_test)

accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, pos_label="spam")

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))

Accuracy: 0.94
Precision: 0.92


# Logistic Regression

# Feature Extraction (Bag-of-Words)
A CountVectorizer is initialized. This is a text feature extraction technique that converts text documents into a matrix of token counts.
X_train is created by applying the fit_transform method of the vectorizer to the train_texts. This step generates a matrix where rows correspond to documents and columns correspond to individual words in the documents. The values in the matrix indicate the frequency of each word in each document.
X_test is created by applying the transform method of the vectorizer to the test_texts. This ensures that the same vocabulary and word-count structure used for training are applied to the test data

In [27]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Multinomial Logistic Regression classifier
The maximum number of iterations for the logistic regression solver is set to 10 using the max_iterations variable.
An instance of LogisticRegression is initialized with the specified parameters: "lbfgs" solver (a type of optimization algorithm), "multinomial" as the multi-class option, and the maximum number of iterations.
The classifier is trained using the fit method with the training feature matrix X_train and the corresponding labels train_labels.

In [33]:
max_iterations = 100  # Set a higher value
clf = LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=max_iterations)
clf.fit(X_train, train_labels)



LogisticRegression(multi_class='multinomial')

# Evaluation
The trained classifier is used to predict labels for the test feature matrix X_test using the predict method. The predictions are stored in the predictions array.
The accuracy and precision scores are calculated using the accuracy_score and precision_score functions from scikit-learn, respectively. The precision is calculated specifically for the "spam" class using the pos_label parameter.
The calculated accuracy and precision scores are printed to the console.

In [37]:
predictions = clf.predict(X_test)
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, pos_label="spam")

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))

Accuracy: 0.94
Precision: 0.89


# Load stop words from file
In this step, the code loads a set of stop words from a file named "stopwords.txt". Stop words are common words like "and", "the", "is", etc., that are often removed from text during preprocessing to reduce noise and improve text analysis.

In [39]:
with open("stopwords.txt", "r") as f:
    stopwords = set(f.read().splitlines())


# Process the text data with stop word removal
The preprocess_text function takes a text input and processes it by performing the following steps:

Tokenizes the text using the spaCy language model (nlp).
Checks if each token's lowercase text is not in the set of stop words. If it's not a stop word, it lemmatizes the token and joins the lemmatized tokens into a string. This effectively removes stop words and performs lemmatization.

In [40]:
def preprocess_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join(token.lemma_ for token in doc if token.text.lower() not in stopwords)
    return lemmatized_text


# Load and preprocess the data from subdirectories
The load_data_from_subdirs function loads and preprocesses the text data from subdirectories within a root directory. It iterates over subdirectories representing different labels (e.g., "spam" or "ham") and reads the text files, processes the text using the preprocess_text function (which includes stop word removal and lemmatization), and appends the processed text and corresponding labels to the texts and labels lists.

In [ ]:
def load_data_from_subdirs(root_dir):
    texts = []
    labels = []
    for label in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label)
        if os.path.isdir(label_path):
            for root, _, files in os.walk(label_path):
                for filename in files:
                    file_path = os.path.join(root, filename)
                    with open(file_path, "r", encoding="latin-1") as file:  
                        text = file.read()
                        processed_text = preprocess_text(text)
                        texts.append(processed_text)
                        labels.append(label)
    return texts, labels

train_texts, train_labels = load_data_from_subdirs("A-2-Corpus-Naive-Bayes-and-Logistic-Regression/train")
test_texts, test_labels = load_data_from_subdirs("A-2-Corpus-Naive-Bayes-and-Logistic-Regression/test")


# Feature Extraction (Bag-of-Words)
A CountVectorizer is initialized to convert the processed text data into a bag-of-words representation. The fit_transform method is applied to the training data (train_texts) to generate the feature matrix X_train, and the transform method is applied to the test data (test_texts) to generate the feature matrix X_test.

In [41]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Multinomial Naive Bayes classifier
A Multinomial Naive Bayes classifier (MultinomialNB) is initialized. It is trained using the training feature matrix X_train and corresponding labels (train_labels). Predictions are made on the test feature matrix X_test, and accuracy and precision scores are calculated for the predictions. The accuracy and precision values are printed to the console, specifically for the Naive Bayes classifier.

In [42]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, train_labels)
nb_predictions = nb_clf.predict(X_test)

nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, pos_label="spam")

print("Naive Bayes Accuracy: {:.2f}".format(nb_accuracy))
print("Naive Bayes Precision: {:.2f}".format(nb_precision))

Naive Bayes Accuracy: 0.94
Naive Bayes Precision: 0.92


# Multinomial Logistic Regression classifier
A Multinomial Logistic Regression classifier (LogisticRegression) is initialized. The classifier is trained using the training feature matrix X_train and corresponding labels (train_labels). Predictions are made on the test feature matrix X_test, and accuracy and precision scores are calculated for these predictions. The accuracy and precision values are printed to the console, specifically for the Logistic Regression classifier.

In [43]:
max_iterations = 100 
logreg_clf = LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=max_iterations)
logreg_clf.fit(X_train, train_labels)
logreg_predictions = logreg_clf.predict(X_test)

logreg_accuracy = accuracy_score(test_labels, logreg_predictions)
logreg_precision = precision_score(test_labels, logreg_predictions, pos_label="spam")

print("Logistic Regression Accuracy: {:.2f}".format(logreg_accuracy))
print("Logistic Regression Precision: {:.2f}".format(logreg_precision))


Logistic Regression Accuracy: 0.94
Logistic Regression Precision: 0.89


In [44]:
pip install -U notebook-as-pdf

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install nbconvert[webpdf]

Note: you may need to restart the kernel to use updated packages.
